# 1 - Multilayer Perceptron For Taxi Prediction

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import copy
import random
import time

# RNG SEED

In [2]:
SEED = 12345

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
class CustomDataset(Dataset):
    def __init__(self, train):
        self.train_data = torch.tensor(pd.read_csv(train).astype('float32').values)
        self.targets = torch.tensor(pd.read_csv("targets_engineered.csv").astype('float32').values)

    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        return self.train_data[idx], self.targets[idx]

The first thing we'll do is load our dataset.

This will automatically download the training set for the MNIST dataset and save it in a folder called `.data`. It will create the folder if it does not exist.

In [4]:
train_data = CustomDataset("train_engineered.csv")

In [5]:
train_data.features

In [6]:
train_data[0]

(tensor([  0.0000,   0.0000,   1.0000,   0.0000,   0.0000,   0.0000, 691.6916,
           0.0000,  60.0000]),
 tensor([330.]))

We can simply check the `len` of the datasets to see how many examples are within each.

In [7]:
print(f'Number of training examples: {len(train_data)}')

Number of training examples: 1710670


In [8]:
VALID_RATIO = 0.95

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

In [9]:
train_data, valid_data = data.random_split(train_data,
                                           [n_train_examples, n_valid_examples])

TEST_RATIO = 0.95
n_valid_examples = int(len(valid_data) * VALID_RATIO)
n_test_examples = len(valid_data) - n_valid_examples
valid_data, test_data = data.random_split(valid_data,
                                           [n_valid_examples, n_test_examples])

In [10]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 1625136
Number of validation examples: 81257
Number of testing examples: 4277


# BATCH SIZE

In [11]:
BATCH_SIZE = 6400

train_iterator = data.DataLoader(train_data,
                                 shuffle=True,
                                 batch_size=BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data,
                                 batch_size=BATCH_SIZE)

test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)

In [12]:
x, y = next(iter(train_iterator))
print(x)
print(y)

tensor([[0.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 6.7432e+02, 4.0000e+00,
         1.8300e+02],
        [0.0000e+00, 1.0000e+00, 0.0000e+00,  ..., 7.5070e+02, 3.0000e+00,
         9.4700e+02],
        [0.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 6.9194e+02, 6.0000e+00,
         2.9100e+02],
        ...,
        [0.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 8.5532e+02, 2.0000e+00,
         7.0500e+02],
        [0.0000e+00, 1.0000e+00, 0.0000e+00,  ..., 7.1717e+02, 2.0000e+00,
         1.1340e+03],
        [0.0000e+00, 1.0000e+00, 0.0000e+00,  ..., 6.6639e+02, 3.0000e+00,
         6.5700e+02]])
tensor([[ 465.],
        [ 480.],
        [ 630.],
        ...,
        [ 780.],
        [1455.],
        [ 735.]])


In [13]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims=[1000, 500, 250, 100, 50], dropout_rates=[0.5, 0.4, 0.3, 0.2, 0.1]):
        super().__init__()

        self.layers = nn.ModuleList()
        dims = [input_dim] + hidden_dims

        for idx in range(len(dims)-1):
            self.layers.append(nn.Linear(dims[idx], dims[idx+1]))
            self.layers.append(nn.BatchNorm1d(dims[idx+1]))
            self.layers.append(nn.Dropout(dropout_rates[idx]))
            self.layers.append(nn.ReLU())

        self.output_fc = nn.Linear(dims[-1], output_dim)

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)

        for layer in self.layers:
            x = layer(x)

        y_pred = self.output_fc(x)

        return y_pred, x


In [14]:
INPUT_DIM = 9
OUTPUT_DIM = 1
HIDDEN_DIMS = [1000, 500, 250, 100, 50]
DROPOUT_RATES = [0.5, 0.4, 0.3, 0.2, 0.1]

model = MLP(INPUT_DIM, OUTPUT_DIM, HIDDEN_DIMS, DROPOUT_RATES)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [16]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 669,751 trainable parameters


# LOSS FUNCTION

In [17]:
criterion = nn.MSELoss()

We then define `device`. This is used to place your model and data on to a GPU, if you have one.

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

We place our model and criterion on to the device by using the `.to` method.

In [19]:
model = model.to(device)
criterion = criterion.to(device)

# OPTIMIZER

In [20]:
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, weight_decay=1e-5)

Next, we'll define a function to calculate the accuracy of our model. This takes the index of the highest value for your prediction and compares it against the actual class label. We then divide how many our model got correct by the amount in the batch to calculate accuracy across the batch.

In [21]:
def calculate_accuracy(y_pred, y):
    '''top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]'''
    return 0 #acc

In [22]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += 0 #acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += 0 #acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

The final step before training is to define a small function to tell us how long an epoch took.

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We're finally ready to train!

During each epoch we calculate the training loss and accuracy, followed by the validation loss and accuracy. We then check if the validation loss achieved is the best validation loss we have seen. If so, we save our model's parameters (called a `state_dict`).

# NUMBER OF EPOCHS

In [25]:
EPOCHS = 100

best_valid_loss = float('inf')
loss_values = []

for epoch in trange(EPOCHS):

    start_time = time.monotonic()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model-r3.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/254 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 971530.404 | Train Acc: 0.00%
	 Val. Loss: 905139.471 |  Val. Acc: 0.00%


Training:   0%|          | 0/254 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch: 99 | Epoch Time: 0m 21s
	Train Loss: 436330.193 | Train Acc: 0.00%
	 Val. Loss: 385035.543 |  Val. Acc: 0.00%


Training:   0%|          | 0/254 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch: 100 | Epoch Time: 0m 21s
	Train Loss: 436073.268 | Train Acc: 0.00%
	 Val. Loss: 384119.197 |  Val. Acc: 0.00%


In [45]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Afterwards, we load our the parameters of the model that achieved the best validation loss and then use this to evaluate our model on the test set.

In [46]:
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 239401.750 | Test Acc: 0.00%


In [47]:
import math 

mse = 239401.750
rmse = math.sqrt(mse)

print(f'Test Loss: {test_loss:.3f}')
print("Test RMSE:", rmse)

Test Loss: 239401.750
Test RMSE: 489.2869812288081


In [36]:
test_data = CustomDataset("test_engineered.csv")
test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE)

df_result = pd.read_csv("sampleSubmission.csv", index_col = False)

model.eval()

with torch.no_grad():

    for (x, y) in tqdm(test_iterator, desc="Evaluating", leave=False):

        x = x.to(device)
        y = y.to(device)

        y_pred, _ = model(x)
        
        df_result['TRAVEL_TIME'] = pd.DataFrame(y_pred.cpu().numpy())

df_result.to_csv("submission.csv", index=False)

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]